## Importing Required Libraries

In [1]:
import torch

from lstm_models import LSTM
from gpu_dataloader import GPUDataset

from torch.utils import tensorboard

import torch.onnx


/Users/macbook/opt/miniconda3/envs/ml_pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading the ML Model to Convert

In [2]:

EPOCH = 'epoch'
LEARNING_RATE = 'learning_rate'
INPUT_SIZE = 'input_size'
HIDDEN_SIZE = 'hidden_size'
NUM_LAYERS = 'num_layers'
NUM_CLASSES = 'num_classes'
MODEL_STATE_DICT = 'model_state_dict'
OPTIMIZER_STATE_DICT = 'optimizer_state_dict'


In [3]:
model_file_path: str = './models/epochs-1-Thu Sep 15 17:34:21 2022'

# model = LSTM()
checkpoint = torch.load('models/epochs-1-Thu Sep 15 17:34:21 2022')
# state_dict = torch.load(model_file_path)
# state_dict

# model.load_state_dict(state_dict)

In [4]:
# checkpoint

In [5]:
writer = tensorboard.SummaryWriter('./models/')

In [6]:
input_size = checkpoint[INPUT_SIZE]
hidden_size = checkpoint[HIDDEN_SIZE]
num_layers = checkpoint[NUM_LAYERS]
num_classes = checkpoint[NUM_CLASSES]
model_dict = checkpoint[MODEL_STATE_DICT]
optimizer_dict = checkpoint[OPTIMIZER_STATE_DICT]

In [7]:
model = LSTM(num_classes, input_size, hidden_size, num_layers, 1)
model.load_state_dict(model_dict)
model

LSTM(
  (lstm): LSTM(19, 2432, batch_first=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc_1): Linear(in_features=2432, out_features=512, bias=True)
  (fc_2): Linear(in_features=512, out_features=256, bias=True)
  (fc_3): Linear(in_features=256, out_features=1, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
)

In [8]:
dataset = GPUDataset(small_df=True)
sample_input = dataset.X
sample_input

tensor([[[ 7.7506e-01, -8.8638e-02,  9.8958e-01,  ..., -4.4614e-01,
          -1.7427e+00, -2.0004e-02]],

        [[ 4.9022e-01,  6.0645e+00,  5.8686e-01,  ..., -4.4614e-01,
           5.7381e-01, -2.0004e-02]],

        [[ 4.8924e-01,  6.0637e+00,  5.9324e-01,  ..., -4.4614e-01,
           5.7381e-01, -2.0004e-02]],

        ...,

        [[ 2.1003e-01,  1.8931e+00, -4.9957e-03,  ..., -4.4614e-01,
           5.7381e-01, -2.0004e-02]],

        [[ 2.3145e-01,  1.9408e+00, -2.1926e-02,  ..., -4.4614e-01,
           5.7381e-01, -2.0004e-02]],

        [[ 2.4433e-01,  1.9058e+00, -7.2713e-03,  ..., -4.4614e-01,
           5.7381e-01, -2.0004e-02]]])

In [9]:
sample_input.flatten()

tensor([ 0.7751, -0.0886,  0.9896,  ..., -0.4461,  0.5738, -0.0200])

In [10]:
writer.add_graph(model, sample_input)

In [12]:
torch.onnx.export(
    model, 
    sample_input,
    './models/lstm.onnx',
    export_params=True,
    input_names=['input'],
    output_names=['output']
)

/Users/macbook/opt/miniconda3/envs/ml_pytorch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:3587: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
/Users/macbook/opt/miniconda3/envs/ml_pytorch/lib/python3.9/site-packages/torch/onnx/_patch_torch.py:67: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1659424435793/work/torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1884.)
  torch._C._jit_pass_onnx_node_shape_type_inference(
/Users/macbook/opt/miniconda3/envs/ml_pytorch/lib/python3.9/site-packages/torch/onnx